## Importação das bibliotecas

In [46]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pyart
import warnings
import math
from mpl_toolkits.basemap import Basemap

%matplotlib inline

### Leitura dos dados

In [6]:
xds = xr.open_dataset('data/sbmn_cappi_20140103_031208.nc')

### Descrição das variáveis

In [8]:
xds.data_vars

Data variables:
    start_time      (time) datetime64[ns] ...
    stop_time       (time) datetime64[ns] ...
    time_bounds     (time, bounds) datetime64[ns] ...
    grid_mapping_0  int32 ...
    VEL             (time, z0, y0, x0) float32 ...
    DBZc            (time, z0, y0, x0) float32 ...
    range           (time, z0, y0, x0) float32 ...
    Coverage        (time, z0, y0, x0) float32 ...

#### Unidade das variáveis

In [9]:
dbz_unit = xds.DBZc.units
x0_unit = xds.x0.units
y0_unit = xds.y0.units

print('Unidade DBCc:   '+dbz_unit+ '\nUnidade de X0:  ' + x0_unit + '\nUnidade de Y0:  ' + y0_unit)

Unidade DBCc:   dBZ
Unidade de X0:  km
Unidade de Y0:  km


#### Agregação dos valores

In [13]:
dbz = xds.DBZc.data
lat = xds.DBZc['lat0'].data
lon = xds.DBZc['lon0'].data

### Calculo do MAXCAPPI

In [70]:
def maxCAPPI(data):
    
    maxC = np.zeros(241*241)-9999
    maxC = np.reshape(maxC,(241,241))
    
    #### Percorrendo matrix 40x
    for k in range(len(data[0])):
        ### Percorre Linhas
        for i in range(len(data[0][k])):
            ### Percorre Colunas
            for j in range(len(data[0][k])):
                #verifica se é diferente de nan
                if math.isnan(data[0][k][i][j]) != True:
                    ### Verifica se i e j são maiores que o anterior
                    if data[0][k][i][j] > maxC[i][j]:
                        maxC[i][j] = data[0][k][i][j]
    
    ## Altera valor -9999 para nan
    for i in range(241):
        for j in range(241):
            if maxC[i][j] == -9999:
                maxC[i][j] = np.float32('0.0')
    return maxC

In [71]:
dados = maxCAPPI(dbz)

### Plot MAXCAPPI

In [56]:
lat0 = lat.mean()
lon0 = lon.mean()

lon0

-59.991997

In [98]:
my_coords = [-3.2, -59.4514]
zoom_scale = 6

levs = np.linspace(0,80,41,endpoint=True)
norm = mpl.colors.BoundaryNorm(levs,256)
cmap = 'pyart_NWSRef'
ax = axes


bbox = [my_coords[0]-zoom_scale,my_coords[0]+zoom_scale,\
        my_coords[1]-zoom_scale,my_coords[1]+zoom_scale]

plt.figure(figsize=(12,6))
m = Basemap(projection='merc',llcrnrlat=bbox[0],urcrnrlat=bbox[1],\
            llcrnrlon=bbox[2],urcrnrlon=bbox[3],lat_ts=10,resolution='i')

#cs = m.pcolormesh(lat0,lon0,dados,norm=norm,cmap=cmap,ax=ax,latlon=False)

m.pcolormesh(lon0,lat0,dados,shading='flat',cmap=plt.cm.jet,latlon=True)

#cs = m.plot(lat0,lon0,dados)


# m.drawcoastlines()
# m.fillcontinents(color='#FAFAFA',lake_color='dodgerblue')

# m.drawparallels(np.arange(bbox[0],bbox[1],(bbox[1]-bbox[0])/5),labels=[1,0,0,0])
# m.drawmeridians(np.arange(bbox[2],bbox[3],(bbox[3]-bbox[2])/5),labels=[0,0,0,1],rotation=45)
# m.drawmapboundary(fill_color='dodgerblue')

# # # # # # Add Colorbar
# # cbar = m.colorbar(cs, location='bottom', pad="10%")
# # cbar.set_label(label)

x,y = m(my_coords[1],my_coords[0])
plt.title("Geographic Amazon")
plt.ylabel('Longitude', labelpad=40)
plt.xlabel('Latitude', labelpad=60)

plt.show()

/ssd/miniconda3/envs/cap394/lib/python3.6/site-packages/ipykernel_launcher.py:15: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  from ipykernel import kernelapp as app


ValueError: 1-d or 2-d longitudes required

<Figure size 864x432 with 0 Axes>

In [75]:
dados

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [68]:
dados2 = np.nan_to_num(dados)

In [69]:
dados2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
np.savetxt("dadosNAN.csv", dados, delimiter=",")